### *NeuralNet_Practice*

In [4]:
#July 2nd, 2023
#Goal:
#Understand Vectorization, activation functions

#Natural Language is how we, humans, exchange ideas and opinions. There are two main mediums for natural language – speech and text. 
#Listening and reading are effortless for a healthy human, but they’re difficult for a machine learning algorithm. That’s why scientists had to come up with Natural Language Processing (NLP).
#What is Natural Language Processing?
#NLP enables computers to process human language and understand meaning and context, along with the associated sentiment and intent behind it, and eventually, use these insights to create something new.
#NLP combines computational linguistics with statistical Machine Learning and Deep Learning models.

# Vectorization is jargon for a classic approach of converting input data from its raw format (i.e. text ) into vectors of real numbers which is the format that ML models support. This approach has been there ever since computers were first built, it has worked wonderfully across various domains, and it’s now used in NLP.
# In Machine Learning, vectorization is a step in feature extraction. The idea is to get some distinct features out of the text for the model to train on, by converting text to numerical vectors.

#https://neptune.ai/blog/vectorization-techniques-in-nlp-guide#:~:text=In%20Machine%20Learning%2C%20vectorization%20is,converting%20text%20to%20numerical%20vectors.

In [5]:
#Refresh Neural Net,  A neural network is a very powerful machine learning mechanism which basically mimics how a human brain learns.
#The activation functions help the network use the important information and suppress the irrelevant data points. 
#Activation functions add non-linearity to the output which enables neural networks to solve non-linear problems.  In other words, a neural network without an activation function is essentially just a linear regression model. 

#Weight decides the influence the input will have on the output while the bias is used to shift the result of activation function towards the positive or negative side.

#vanishing gradient, no gradient no learning

In [ ]:
# There are other activation functions that we don't have to priortize - Linear and Step

In [6]:
#Sigmoid {0 & 1}

import math 

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [7]:
sigmoid(100)

1.0

In [8]:
sigmoid(400)

1.0

In [9]:
sigmoid(40)

1.0

In [11]:
#Tanh {-1 and 1}

def tanh(x):
  return (math.exp(x) - math.exp(-x)) / (math.exp(x) + math.exp(-x))

In [12]:
tanh(1)

0.7615941559557649

In [14]:
#Relu

def relu(x):
    return max(0,x)

In [15]:
relu(50)

50

In [16]:
#Leaky ReLU

def leaky_relu(x):
    return max(0.1*x,x)

In [19]:
leaky_relu(-5)

-0.5

In [3]:
#July 3rd
#backprop/forwardprop

#### Back/Forward propagation

In [ ]:
#After the forward pass and cost calculation (error/loss), Backpropagation takes place to get the delta. 
#For calculating delta, it applies partial derivative and it starts from output to the last layer. And, 
#once delta is calculated for the last layer, below formula is applied to get the new weight.

In [1]:
import tensorflow as tf

In [3]:
import os

os.getcwd()

'c:\\Users\\DELL\\OneDrive\\Documents\\ML_Project\\NN_Learnings'

In [11]:
#Implementing Resnet

from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

num_classes = 2
resnet_weights_path = 'c:\\Users\\DELL\\OneDrive\\Documents\\ML_Project\\NN_Learnings\\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg'))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.load_weights(resnet_weights_path, by_name=True, skip_mismatch=True)
my_new_model.summary()

94765736/94765736 [==============================] - 2s 0us/step
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [12]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 225
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        'c:\\Users\\DELL\\ML_Project\\NN_Learnings\\train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'c:\\Users\\DELL\\ML_Project\\NN_Learnings\\val',
        target_size=(image_size, image_size),
        batch_size=20,
        class_mode='categorical')

Found 72 images belonging to 2 classes.


Found 20 images belonging to 2 classes.


In [30]:
my_new_model.fit(
        train_generator,
        steps_per_epoch=6,
        validation_data=validation_generator,
        validation_steps=1)

6/6 [==============================] - 12s 2s/step - loss: 0.4061 - accuracy: 0.7917 - val_loss: 0.0817 - val_accuracy: 1.0000


#### Optimization using Keras Tuner

In [26]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-26-249b82b79d59>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [23]:
fashion_mnist=keras.datasets.fashion_mnist

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [24]:
train_images=train_images/255.0
test_images=test_images/255.0

train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [25]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [27]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [28]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 12m 24s]
val_accuracy: 0.9178333282470703

Best val_accuracy So Far: 0.9178333282470703
Total elapsed time: 00h 44m 35s
INFO:tensorflow:Oracle triggered exit


In [34]:
model=tuner_search.get_best_models(num_models=1)[0]

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4738826 (18.08 MB)
Trainable params: 4738826 (18.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=8)

Epoch 9/10
1688/1688 [==============================] - 210s 125ms/step - loss: 0.0415 - accuracy: 0.9851 - val_loss: 0.3700 - val_accuracy: 0.9157
Epoch 10/10
1688/1688 [==============================] - 204s 121ms/step - loss: 0.0285 - accuracy: 0.9902 - val_loss: 0.4382 - val_accuracy: 0.9188
